In [158]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
from sklearn.preprocessing import StandardScaler

In [159]:
dataset=pd.read_csv("computer_dataset_clean.csv")
dataset.isna().sum()
dataset["Weight"].fillna("0.0",inplace=True)

In [160]:
dataset.head()

,Unnamed: 0,Company,TypeName,Inches,ScreenResolution,Cpu,Ram,Memory,Gpu,OpSys,Weight,Price
0,0,Apple,Ultrabook,13.3,IPS Panel Retina Display 2560x1600,Intel Core i5 2.3GHz,8GB,128GB SSD,Intel Iris Plus Graphics 640,macOS,1.37kg,71378.6832
1,1,Apple,Ultrabook,13.3,1440x900,Intel Core i5 1.8GHz,8GB,128GB Flash Storage,Intel HD Graphics 6000,macOS,1.34kg,47895.5232
2,2,HP,Notebook,15.6,Full HD 1920x1080,Intel Core i5 7200U 2.5GHz,8GB,256GB SSD,Intel HD Graphics 620,N.A,1.86kg,30636.0000
3,3,Apple,Ultrabook,15.4,IPS Panel Retina Display 2880x1800,Intel Core i7 2.7GHz,16GB,512GB SSD,AMD Radeon Pro 455,macOS,1.83kg,135195.3360
4,4,Apple,Ultrabook,13.3,IPS Panel Retina Display 2560x1600,Intel Core i5 3.1GHz,8GB,256GB SSD,Intel Iris Plus Graphics 650,macOS,1.37kg,96095.8080


In [76]:
dataset.isna().sum()

Unnamed: 0          0
Company             0
TypeName            0
Inches              0
ScreenResolution    0
Cpu                 0
Ram                 0
Memory              0
Gpu                 0
OpSys               0
Weight              0
Price               0
dtype: int64

In [124]:
dataset.drop('Unnamed: 0',axis=1,inplace=True)

In [126]:
dataset

,Company,TypeName,Inches,ScreenResolution,Cpu,Ram,Memory,Gpu,OpSys,Weight,Price
0,1,4,13.3,23,65,9,4,58,8,40,71378.6832
1,1,4,13.3,1,63,9,2,51,8,37,47895.5232
2,7,3,15.6,8,74,9,16,53,4,76,30636.0000
3,1,4,15.4,25,85,1,29,9,8,73,135195.3360
4,1,4,13.3,23,67,9,16,59,8,40,96095.8080
...,...,...,...,...,...,...,...,...,...,...,...
1238,2,3,15.6,0,89,6,26,65,5,116,38378.6496
1239,10,0,14.0,13,89,6,4,47,5,80,33992.6400
1240,10,0,13.3,19,89,1,29,47,5,43,79866.7200
1241,10,3,14.0,0,34,4,35,40,5,58,12201.1200


# 1 drop categorical data

In [127]:
lis_col=['Company', 'TypeName', 'ScreenResolution',
       'Cpu', 'Ram', 'Memory', 'Gpu', 'OpSys', 'Weight']
lis_col
from sklearn.preprocessing import LabelEncoder

encoder = LabelEncoder()

for col in lis_col:  # Replace with actual column names
    dataset[col] = encoder.fit_transform(dataset[col])
    

In [128]:
independent=dataset.drop("Price",axis=1)
dependent=dataset["Price"]

# 2 first we select n features using RFE

In [129]:
from sklearn.linear_model import LinearRegression
from sklearn.svm import SVR
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.feature_selection import RFE

def rfefeature(independent,dependent,n):
    rfelist=[]
    #RFE has many models cos for each model the n no of features will be different
    # it uses models as the evaluation metrics for deciding n,like chis_core for kbest.
    reg_model=LinearRegression(fit_intercept=True)
    svr=SVR(kernel="linear")
    RF=RandomForestRegressor(n_estimators=3,criterion="squared_error",random_state=0)
    DT=DecisionTreeRegressor(criterion="squared_error")
    
    rfemodellist=[reg_model,svr,RF,DT]
    
    for i in  rfemodellist:
        print(i)
        model=RFE(estimator=i, n_features_to_select=n)
        logrfe_fit=model.fit(independent,dependent) 
        
        logrfe_feature=logrfe_fit.transform(independent)
        selected_feature_names = independent.columns[logrfe_fit.support_]
        rfelist.append(logrfe_feature)
    return rfelist,selected_feature_names

In [144]:
rfelist,selected_feature_names=rfefeature(independent,dependent,6)

LinearRegression()
SVR(kernel='linear')
RandomForestRegressor(n_estimators=3, random_state=0)
DecisionTreeRegressor()


In [145]:
rfelist

[array([[ 1., 23., 65.,  9.,  4.,  8.],
        [ 1.,  1., 63.,  9.,  2.,  8.],
        [ 7.,  8., 74.,  9., 16.,  4.],
        ...,
        [10., 19., 89.,  1., 29.,  5.],
        [10.,  0., 34.,  4., 35.,  5.],
        [ 7.,  0., 89.,  8., 10.,  5.]]),
 array([[ 1., 23., 65.,  9.,  4.,  8.],
        [ 1.,  1., 63.,  9.,  2.,  8.],
        [ 7.,  8., 74.,  9., 16.,  4.],
        ...,
        [10., 19., 89.,  1., 29.,  5.],
        [10.,  0., 34.,  4., 35.,  5.],
        [ 7.,  0., 89.,  8., 10.,  5.]]),
 array([[  4.,  65.,   9.,   4.,  58.,  40.],
        [  4.,  63.,   9.,   2.,  51.,  37.],
        [  3.,  74.,   9.,  16.,  53.,  76.],
        ...,
        [  0.,  89.,   1.,  29.,  47.,  43.],
        [  3.,  34.,   4.,  35.,  40.,  58.],
        [  3.,  89.,   8.,  10.,  21., 107.]]),
 array([[ 23.,  65.,   9.,   4.,  58.,  40.],
        [  1.,  63.,   9.,   2.,  51.,  37.],
        [  8.,  74.,   9.,  16.,  53.,  76.],
        ...,
        [ 19.,  89.,   1.,  29.,  47.,  43.],
  

In [146]:
selected_feature_names

Index(['ScreenResolution', 'Cpu', 'Ram', 'Memory', 'Gpu', 'Weight'], dtype='object')

In [147]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
def split_scaler(independent,dependent):
    x_train,x_test,y_train,y_test=train_test_split(independent,dependent,test_size=.25,random_state=0)
    sc=StandardScaler()
    x_train=sc.fit_transform(x_train)
    x_test=sc.transform(x_test)
    return x_train,x_test,y_train,y_test


In [148]:
for i in rfelist:
    x_train,x_test,y_train,y_test=split_scaler(i,dependent) 

In [149]:
accsvm=[]
accs_MULTILINEAR=[]
accRF=[]
accDEC_tree=[]

In [150]:
def r2_score(model,x_test,y_test):
    from sklearn.metrics import r2_score
    y_predict=model.predict(x_test)
    r_score=r2_score(y_test,y_predict)
    return r_score,model

In [151]:
def lin_reg(x_train,y_train,x_test,y_test):
    from sklearn.linear_model import LinearRegression
    from sklearn.model_selection import GridSearchCV
    model=LinearRegression()
    param_grid={"fit_intercept":[True,False]}
    grid_Search= GridSearchCV(model,param_grid,verbose=3,n_jobs=1,refit=True)
    grid_Search.fit(x_train,y_train)
    r_score,model=r2_score(grid_Search,x_test,y_test)
    return r_score,model

In [152]:
def svm(x_train,y_train,x_test,y_test):
    from sklearn.svm import SVR
    from sklearn.model_selection import GridSearchCV
    model=SVR()
    param_grid={"kernel":["poly","sigmoid","rbf"],"C":[0.1,1,10]}
    grid_search = GridSearchCV(model,param_grid,verbose=3,n_jobs=1,refit=True)
    grid_search.fit(x_train,y_train)
    R2_score,model=r2_score(grid_search,x_test,y_test)
    return R2_score,model

In [153]:
def decision_tree(x_train,y_train,x_test,y_test):
    from sklearn.tree import DecisionTreeRegressor
    from sklearn.model_selection import GridSearchCV
    model=DecisionTreeRegressor()
    param_grid={"criterion":["friedman_mse","squared_error"],"splitter":["random","best"],"max_depth":[None,5,10]}
    grid_search=GridSearchCV(model,param_grid,verbose=3,n_jobs=1,refit=True)
    grid_search.fit(x_train,y_train)
    R2_score,model=r2_score(grid_search,x_test,y_test)
    return R2_score,model

In [154]:
def random_forest(x_train,y_train,x_test,y_test):
    from sklearn.ensemble import RandomForestRegressor
    from sklearn.model_selection import GridSearchCV
    model=RandomForestRegressor()
    param_grid={"n_estimators": [100, 200, 500],"criterion":["friedman_mse","squared_error"],"max_depth":[None,5,10]}
    grid_search=GridSearchCV(model,param_grid,verbose=3,n_jobs=1,refit=True)
    grid_search.fit(x_train,y_train)
    R2_score,model=r2_score(grid_search,x_test,y_test)
    return R2_score,model

In [155]:
def RFE_Features(accs_MULTILINEAR,accsvm,accDEC_tree,accRF):
    dataframe=pd.DataFrame(index=["multilinear","svm","decisiontree","RF"],columns=["multilinear","svm","decisiontree","RF"])
    for number,idex in enumerate(dataframe.index):
                           dataframe["multilinear"][idex]=accs_MULTILINEAR[number]
                           dataframe["svm"][idex]=accsvm[number]
                           dataframe["decisiontree"][idex]=accDEC_tree[number]
                           dataframe["RF"][idex]=accRF[number]
    return dataframe

In [156]:
for i in rfelist:
    x_train,x_test,y_train,y_test=split_scaler(i,dependent)
    R2_score,model=lin_reg(x_train,y_train,x_test,y_test)
    accs_MULTILINEAR.append(R2_score)
    R2_score,model=svm(x_train,y_train,x_test,y_test)
    accsvm.append(R2_score)
    R2_score,model=decision_tree(x_train,y_train,x_test,y_test)
    accDEC_tree.append(R2_score)
    R2_score,model=random_forest(x_train,y_train,x_test,y_test)
    accRF.append(R2_score)
    
result1=RFE_Features(accs_MULTILINEAR,accsvm,accDEC_tree,accRF)

Fitting 5 folds for each of 2 candidates, totalling 10 fits
[CV 1/5] END ................fit_intercept=True;, score=0.493 total time=   0.0s
[CV 2/5] END ................fit_intercept=True;, score=0.459 total time=   0.0s
[CV 3/5] END ................fit_intercept=True;, score=0.449 total time=   0.0s
[CV 4/5] END ................fit_intercept=True;, score=0.470 total time=   0.0s
[CV 5/5] END ................fit_intercept=True;, score=0.458 total time=   0.0s
[CV 1/5] END ..............fit_intercept=False;, score=-3.019 total time=   0.0s
[CV 2/5] END ..............fit_intercept=False;, score=-2.419 total time=   0.0s
[CV 3/5] END ..............fit_intercept=False;, score=-3.198 total time=   0.0s
[CV 4/5] END ..............fit_intercept=False;, score=-3.765 total time=   0.0s
[CV 5/5] END ..............fit_intercept=False;, score=-2.904 total time=   0.0s
Fitting 5 folds for each of 9 candidates, totalling 45 fits
[CV 1/5] END ...............C=0.1, kernel=poly;, score=-0.049 total ti

[CV 5/5] END criterion=squared_error, max_depth=None, splitter=best;, score=0.513 total time=   0.0s
[CV 1/5] END criterion=squared_error, max_depth=5, splitter=random;, score=0.555 total time=   0.0s
[CV 2/5] END criterion=squared_error, max_depth=5, splitter=random;, score=0.605 total time=   0.0s
[CV 3/5] END criterion=squared_error, max_depth=5, splitter=random;, score=0.589 total time=   0.0s
[CV 4/5] END criterion=squared_error, max_depth=5, splitter=random;, score=0.567 total time=   0.0s
[CV 5/5] END criterion=squared_error, max_depth=5, splitter=random;, score=0.530 total time=   0.0s
[CV 1/5] END criterion=squared_error, max_depth=5, splitter=best;, score=0.555 total time=   0.0s
[CV 2/5] END criterion=squared_error, max_depth=5, splitter=best;, score=0.664 total time=   0.0s
[CV 3/5] END criterion=squared_error, max_depth=5, splitter=best;, score=0.597 total time=   0.0s
[CV 4/5] END criterion=squared_error, max_depth=5, splitter=best;, score=0.590 total time=   0.0s
[CV 5/5

[CV 1/5] END criterion=squared_error, max_depth=5, n_estimators=100;, score=0.635 total time=   0.2s
[CV 2/5] END criterion=squared_error, max_depth=5, n_estimators=100;, score=0.650 total time=   0.2s
[CV 3/5] END criterion=squared_error, max_depth=5, n_estimators=100;, score=0.668 total time=   0.2s
[CV 4/5] END criterion=squared_error, max_depth=5, n_estimators=100;, score=0.638 total time=   0.2s
[CV 5/5] END criterion=squared_error, max_depth=5, n_estimators=100;, score=0.632 total time=   0.2s
[CV 1/5] END criterion=squared_error, max_depth=5, n_estimators=200;, score=0.632 total time=   0.4s
[CV 2/5] END criterion=squared_error, max_depth=5, n_estimators=200;, score=0.653 total time=   0.4s
[CV 3/5] END criterion=squared_error, max_depth=5, n_estimators=200;, score=0.665 total time=   0.5s
[CV 4/5] END criterion=squared_error, max_depth=5, n_estimators=200;, score=0.641 total time=   0.4s
[CV 5/5] END criterion=squared_error, max_depth=5, n_estimators=200;, score=0.629 total tim

[CV 1/5] END criterion=friedman_mse, max_depth=10, splitter=best;, score=0.626 total time=   0.0s
[CV 2/5] END criterion=friedman_mse, max_depth=10, splitter=best;, score=0.627 total time=   0.0s
[CV 3/5] END criterion=friedman_mse, max_depth=10, splitter=best;, score=0.616 total time=   0.0s
[CV 4/5] END criterion=friedman_mse, max_depth=10, splitter=best;, score=0.541 total time=   0.0s
[CV 5/5] END criterion=friedman_mse, max_depth=10, splitter=best;, score=0.567 total time=   0.0s
[CV 1/5] END criterion=squared_error, max_depth=None, splitter=random;, score=0.510 total time=   0.0s
[CV 2/5] END criterion=squared_error, max_depth=None, splitter=random;, score=0.587 total time=   0.0s
[CV 3/5] END criterion=squared_error, max_depth=None, splitter=random;, score=0.561 total time=   0.0s
[CV 4/5] END criterion=squared_error, max_depth=None, splitter=random;, score=0.517 total time=   0.0s
[CV 5/5] END criterion=squared_error, max_depth=None, splitter=random;, score=0.451 total time=   

[CV 2/5] END criterion=squared_error, max_depth=None, n_estimators=100;, score=0.715 total time=   0.4s
[CV 3/5] END criterion=squared_error, max_depth=None, n_estimators=100;, score=0.658 total time=   0.3s
[CV 4/5] END criterion=squared_error, max_depth=None, n_estimators=100;, score=0.681 total time=   0.4s
[CV 5/5] END criterion=squared_error, max_depth=None, n_estimators=100;, score=0.685 total time=   0.3s
[CV 1/5] END criterion=squared_error, max_depth=None, n_estimators=200;, score=0.674 total time=   0.7s
[CV 2/5] END criterion=squared_error, max_depth=None, n_estimators=200;, score=0.722 total time=   0.7s
[CV 3/5] END criterion=squared_error, max_depth=None, n_estimators=200;, score=0.662 total time=   0.7s
[CV 4/5] END criterion=squared_error, max_depth=None, n_estimators=200;, score=0.686 total time=   0.8s
[CV 5/5] END criterion=squared_error, max_depth=None, n_estimators=200;, score=0.695 total time=   0.8s
[CV 1/5] END criterion=squared_error, max_depth=None, n_estimato

[CV 3/5] END ..............C=10, kernel=sigmoid;, score=0.024 total time=   0.0s
[CV 4/5] END ..............C=10, kernel=sigmoid;, score=0.001 total time=   0.0s
[CV 5/5] END .............C=10, kernel=sigmoid;, score=-0.004 total time=   0.0s
[CV 1/5] END .................C=10, kernel=rbf;, score=-0.031 total time=   0.0s
[CV 2/5] END .................C=10, kernel=rbf;, score=-0.024 total time=   0.0s
[CV 3/5] END ..................C=10, kernel=rbf;, score=0.011 total time=   0.0s
[CV 4/5] END .................C=10, kernel=rbf;, score=-0.014 total time=   0.0s
[CV 5/5] END .................C=10, kernel=rbf;, score=-0.018 total time=   0.0s
Fitting 5 folds for each of 12 candidates, totalling 60 fits
[CV 1/5] END criterion=friedman_mse, max_depth=None, splitter=random;, score=0.303 total time=   0.0s
[CV 2/5] END criterion=friedman_mse, max_depth=None, splitter=random;, score=0.549 total time=   0.0s
[CV 3/5] END criterion=friedman_mse, max_depth=None, splitter=random;, score=0.491 tota

[CV 5/5] END criterion=friedman_mse, max_depth=None, n_estimators=500;, score=0.750 total time=   2.1s
[CV 1/5] END criterion=friedman_mse, max_depth=5, n_estimators=100;, score=0.707 total time=   0.2s
[CV 2/5] END criterion=friedman_mse, max_depth=5, n_estimators=100;, score=0.719 total time=   0.2s
[CV 3/5] END criterion=friedman_mse, max_depth=5, n_estimators=100;, score=0.704 total time=   0.2s
[CV 4/5] END criterion=friedman_mse, max_depth=5, n_estimators=100;, score=0.711 total time=   0.2s
[CV 5/5] END criterion=friedman_mse, max_depth=5, n_estimators=100;, score=0.711 total time=   0.2s
[CV 1/5] END criterion=friedman_mse, max_depth=5, n_estimators=200;, score=0.721 total time=   0.7s
[CV 2/5] END criterion=friedman_mse, max_depth=5, n_estimators=200;, score=0.723 total time=   0.5s
[CV 3/5] END criterion=friedman_mse, max_depth=5, n_estimators=200;, score=0.701 total time=   0.5s
[CV 4/5] END criterion=friedman_mse, max_depth=5, n_estimators=200;, score=0.709 total time=   0.

[CV 1/5] END ............C=0.1, kernel=sigmoid;, score=-0.049 total time=   0.0s
[CV 2/5] END ............C=0.1, kernel=sigmoid;, score=-0.041 total time=   0.0s
[CV 3/5] END ............C=0.1, kernel=sigmoid;, score=-0.003 total time=   0.0s
[CV 4/5] END ............C=0.1, kernel=sigmoid;, score=-0.031 total time=   0.0s
[CV 5/5] END ............C=0.1, kernel=sigmoid;, score=-0.034 total time=   0.0s
[CV 1/5] END ................C=0.1, kernel=rbf;, score=-0.049 total time=   0.0s
[CV 2/5] END ................C=0.1, kernel=rbf;, score=-0.041 total time=   0.0s
[CV 3/5] END ................C=0.1, kernel=rbf;, score=-0.004 total time=   0.0s
[CV 4/5] END ................C=0.1, kernel=rbf;, score=-0.031 total time=   0.0s
[CV 5/5] END ................C=0.1, kernel=rbf;, score=-0.034 total time=   0.0s
[CV 1/5] END .................C=1, kernel=poly;, score=-0.043 total time=   0.0s
[CV 2/5] END .................C=1, kernel=poly;, score=-0.036 total time=   0.0s
[CV 3/5] END ...............

[CV 1/5] END criterion=friedman_mse, max_depth=None, n_estimators=100;, score=0.728 total time=   0.4s
[CV 2/5] END criterion=friedman_mse, max_depth=None, n_estimators=100;, score=0.760 total time=   0.4s
[CV 3/5] END criterion=friedman_mse, max_depth=None, n_estimators=100;, score=0.738 total time=   0.4s
[CV 4/5] END criterion=friedman_mse, max_depth=None, n_estimators=100;, score=0.776 total time=   0.3s
[CV 5/5] END criterion=friedman_mse, max_depth=None, n_estimators=100;, score=0.725 total time=   0.5s
[CV 1/5] END criterion=friedman_mse, max_depth=None, n_estimators=200;, score=0.730 total time=   0.9s
[CV 2/5] END criterion=friedman_mse, max_depth=None, n_estimators=200;, score=0.765 total time=   1.0s
[CV 3/5] END criterion=friedman_mse, max_depth=None, n_estimators=200;, score=0.738 total time=   0.9s
[CV 4/5] END criterion=friedman_mse, max_depth=None, n_estimators=200;, score=0.781 total time=   0.9s
[CV 5/5] END criterion=friedman_mse, max_depth=None, n_estimators=200;, s

[CV 2/5] END criterion=squared_error, max_depth=10, n_estimators=200;, score=0.759 total time=   0.7s
[CV 3/5] END criterion=squared_error, max_depth=10, n_estimators=200;, score=0.731 total time=   0.7s
[CV 4/5] END criterion=squared_error, max_depth=10, n_estimators=200;, score=0.775 total time=   0.7s
[CV 5/5] END criterion=squared_error, max_depth=10, n_estimators=200;, score=0.735 total time=   0.7s
[CV 1/5] END criterion=squared_error, max_depth=10, n_estimators=500;, score=0.740 total time=   1.8s
[CV 2/5] END criterion=squared_error, max_depth=10, n_estimators=500;, score=0.760 total time=   1.8s
[CV 3/5] END criterion=squared_error, max_depth=10, n_estimators=500;, score=0.733 total time=   1.9s
[CV 4/5] END criterion=squared_error, max_depth=10, n_estimators=500;, score=0.777 total time=   1.9s
[CV 5/5] END criterion=squared_error, max_depth=10, n_estimators=500;, score=0.732 total time=   2.0s


In [143]:
result1# result1 for 5

,multilinear,svm,decisiontree,RF
multilinear,0.483764,0.003709,0.645266,0.704293
svm,0.464492,-0.00111,0.656775,0.719157
decisiontree,0.437713,0.009306,0.698234,0.766864
RF,0.437713,0.009306,0.698234,0.762713


In [157]:
# result1 for 6
result1

,multilinear,svm,decisiontree,RF
multilinear,0.493181,-0.010327,0.653241,0.740447
svm,0.493181,-0.010327,0.636981,0.723693
decisiontree,0.439321,-0.006125,0.679933,0.768693
RF,0.478035,-0.004851,0.702229,0.777304


In [93]:
result1# result1 for 8

,multilinear,svm,decisiontree,RF
multilinear,0.499698,-0.014414,0.690074,0.787705
svm,0.510782,-0.015997,0.713586,0.798032
decisiontree,0.487839,-0.019988,0.688745,0.77521
RF,0.487602,-0.020897,0.69511,0.776036
